In [3]:
import pandas as pd
import numpy as np
import cPickle as pickle
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#Downsamplers imports - prototype selection - Cleaning techniques
from imblearn.under_sampling import TomekLinks

#Downsamplers imports - prototype selection - controlled
from imblearn.under_sampling import RandomUnderSampler

#ML framework imports
from sklearn.metrics import auc, roc_auc_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import scale

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

ABSOLUTE_NEGATIVES = True
FILTER_DOMAIN = False

In [4]:
curr_dir = !pwd
input_path = curr_dir[0]+"/domains_similarity/filtered_features_table/"
filename = "positions_features_mediode_filter_01.25.18.csv"

#input_path = curr_dir[0]+"/../9.Features_exploration/binding_df/10/"
#filename = "positions_features_01.25.18.csv"

bind_scores_num = 10

#Features table
features_all = pd.read_csv(input_path+filename, sep='\t', index_col=0)
features_cols = features_all.columns[1:-bind_scores_num] #removing binding scores and domain name
ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite"]
print "all samples positions #: "+str(features_all.shape[0])

#lignd binding domains dictionary
with open(curr_dir[0]+"/ligands_negatives_domains_dict.pik", 'rb') as handle:
    negatives_dict = pickle.load(handle)

all samples positions #: 38944


### Create dataset of negatives

In [5]:
def filter_to_ligand_binding_domains(use_max_binding_score):
    
    ligands_negatives_df = {}
    for ligand in ligands:
        
        ligands_negatives_df[ligand] = pd.DataFrame()
        for domain in negatives_dict[ligand].keys():
            if domain == 'negatives' or domain == 'domains':
                continue
            domain_all = features_all.loc[features_all.loc[:,"domain_name"] == domain,:]
            
            #In case this domain was previously filtered
            if len(domain_all) == 0:
                continue
            
            if (use_max_binding_score):
                ligands_negatives_df[ligand] = pd.concat([ligands_negatives_df[ligand],domain_all.loc[domain_all.loc[:,"max_binding_score"] == 0,:]])
            else:
                ligand_bind_str = ligand+"_binding_score"
                ligands_negatives_df[ligand] = pd.concat([ligands_negatives_df[ligand],domain_all.loc[domain_all.loc[:,ligand_bind_str] == 0,:]])
        
    #Handeling the ligand "all_ligands"
    all_ligands_negatives_df = pd.concat([ligands_negatives_df["dna"], ligands_negatives_df["dnabase"], ligands_negatives_df["dnabackbone"], ligands_negatives_df["rna"], ligands_negatives_df["rnabase"], 
                                 ligands_negatives_df["rnabackbone"], ligands_negatives_df["ion"], ligands_negatives_df["peptide"], ligands_negatives_df["metabolite"]])
    all_ligands_negatives_df = all_ligands_negatives_df.drop_duplicates()
    #Filter to just positions with max. binding score = 0
    all_ligands_negatives_df = all_ligands_negatives_df[all_ligands_negatives_df["max_binding_score"] == 0]
    ligands_negatives_df["all_ligands"] = all_ligands_negatives_df
    
    #Leaving just the features columns
    for ligand in ligands_negatives_df.keys():   
        ligands_negatives_df[ligand] = ligands_negatives_df[ligand]
        print(ligand+" non-binding #:"+str(len(ligands_negatives_df[ligand])))
    
    return ligands_negatives_df

In [6]:
def negatives_by_binding_score(use_max_binding_score):
    
    ligands_negatives_df = {}
    for ligand in ligands:
        
        if use_max_binding_score:
            ligand_bind_str = "max_binding_score"
        else:
            ligand_bind_str = ligand+"_binding_score"
        
        ligands_negatives_df[ligand] = features_all[features_all[ligand_bind_str] == 0]
        ligands_negatives_df[ligand] = ligands_negatives_df[ligand]
        print(ligand+" non-binding #:"+str(len(ligands_negatives_df[ligand])))
        
    #Handeling the ligand "all_ligands"
    ligands_negatives_df["all_ligands"] = features_all[features_all["max_binding_score"] == 0]
    ligands_negatives_df["all_ligands"] = ligands_negatives_df["all_ligands"]
    print("all_ligands non-binding #:"+str(len(ligands_negatives_df["all_ligands"])))
    
    return ligands_negatives_df

In [7]:
#Create negatives datasets
if FILTER_DOMAIN:
    if ABSOLUTE_NEGATIVES:
        ligands_negatives_df = filter_to_ligand_binding_domains(True)
    else:
        ligands_negatives_df = filter_to_ligand_binding_domains(False)
else:
    if ABSOLUTE_NEGATIVES:
        ligands_negatives_df = negatives_by_binding_score(True)
    else:
        ligands_negatives_df = negatives_by_binding_score(False)

dna non-binding #:27191
dnabase non-binding #:27191
dnabackbone non-binding #:27191
rna non-binding #:27191
rnabase non-binding #:27191
rnabackbone non-binding #:27191
peptide non-binding #:27191
ion non-binding #:27191
metabolite non-binding #:27191
all_ligands non-binding #:27191


### Create dataset of positives

In [8]:
bind_th = 0.1
ligands_features_df = {}

for ligand in ligands:
    score_col_str = ligand+"_binding_score"
    ligand_binding_df = features_all[features_all[score_col_str] >= bind_th]
    print ligand+" #: "+str(ligand_binding_df.shape[0])
    ligands_features_df[ligand] = ligand_binding_df

dna #: 501
dnabase #: 193
dnabackbone #: 408
rna #: 433
rnabase #: 224
rnabackbone #: 308
peptide #: 1496
ion #: 1093
metabolite #: 1525


In [9]:
all_ligands_features_df = pd.concat([ligands_features_df["dna"], ligands_features_df["dnabase"], ligands_features_df["dnabackbone"], ligands_features_df["rna"], ligands_features_df["rnabase"], 
                                     ligands_features_df["rnabackbone"], ligands_features_df["ion"], ligands_features_df["peptide"], ligands_features_df["metabolite"]])
all_ligands_features_df = all_ligands_features_df.drop_duplicates()
print "all_ligands #: "+str(all_ligands_features_df.shape[0])
ligands_features_df["all_ligands"] = all_ligands_features_df

all_ligands #: 4518


### Models

In [11]:
classifiers = {}
classifiers["RF"] = RandomForestRegressor(n_estimators=1000, random_state=0, n_jobs=-1)

In [12]:
downsamplers = {}
downsamplers["TomekLinks"] = TomekLinks(random_state=0)
downsamplers["RandomUnderSampler"] = RandomUnderSampler(random_state=0)

### Test model

In [75]:
def test_model(pred_dict, auc_dict, auprc_dict, ligand_bind_features, ligand_negatives_features, ligand_name, downsample_method, features=[]):
    """
    Test different models in 10-folds cross-validation.
    """
    
    #Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
        
    #Arranging the features table by the CV order, for each model
    features_pred_dfs = dict.fromkeys(classifiers.keys())
    
    models_req_scaling = ["SVM", "KNN"]
    
    for classifier in classifiers.keys():
        model = classifiers[classifier]
        features_pred_dfs[classifier] = pd.DataFrame()
        
        #Create X and y and remove domain names
        X = pd.concat([ligand_bind_features.iloc[:,features], ligand_negatives_features.iloc[:,features]])
        X = X.drop('domain_name', 1)
        cols = X.columns
        
        if (classifier in models_req_scaling):
            idx = X.index
            cols = X.columns
            X = pd.DataFrame(scale(X)) #Is z-scoring the data needed?
            X.index = idx #Restoring indices after scaling
            X.columns = cols
            
        y = [1] * ligand_bind_features.shape[0]
        y.extend([0] * ligand_negatives_features.shape[0])
        y = np.array(y)

        binding_skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
        pred_idx = 1
        
        #Metrics
        exp_var = []
        mse = []
        r2 = []
        
        for train_index, test_index in binding_skf.split(X, y):
            print "fold #: "+str(pred_idx)
            X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
            y_train, y_test = y[train_index], y[test_index]

            #Down-sample negative examples to have balanced classes
            if (downsample_method == "NoDown"):
                X_train_sampled = X_train
                y_train_sampled = y_train
            else:
                if (downsample_method == "InstanceHardnessThreshold"):
                    downsampler = downsamplers[downsample_method][classifier]
                else:
                    downsampler = downsamplers[downsample_method]

                X_train_sampled, y_train_sampled = downsampler.fit_sample(X_train, y_train)
                
            # Adjust x/y train/test to be the continuous binding scores
            included_features_idx = [c in features_cols for c in cols]
            ligand_bind_idx = ligand_name+"_binding_score" == cols
            
            y_train_sampled = X_train_sampled[:,ligand_bind_idx]
            y_train_sampled = np.array(y_train_sampled).ravel()
            X_train_sampled = X_train_sampled[:,included_features_idx]
            
            y_test = X_test.loc[:,ligand_bind_idx]
            y_test = np.array(y_test).ravel()
            X_test = X_test.loc[:,included_features_idx]
            
            #fit to training data
            model = classifiers[classifier]
            model.fit(X_train_sampled, y_train_sampled)
            probs_list = []

            #probs = model.predict(X_test)
            #probs_list = probs
            
            pred_list = model.predict(X_test)
            
            #for l in probs:
            #    probs_list.append(l[1])
                
            pred_dict["obs"].extend(y_test)
            pred_dict["prob"].extend(pred_list)
            fold_list = [pred_idx] * len(pred_list)
            pred_dict["fold"].extend(fold_list)

            model_list = [classifier] * len(pred_list)
            pred_dict["model"].extend(model_list)
            
            #Update metrics
            e = explained_variance_score(y_test, pred_list)
            exp_var.append(e)
            
            m = mean_squared_error(y_test, pred_list)
            mse.append(m)
            
            r = r2_score(y_test, pred_list)
            r2.append(r)
            

            #Update auc auprc dictionaries
            #auc_dict[classifier].append(roc_auc_score(y_test, probs_list))
            #precision, recall, _ = precision_recall_curve(y_test, probs[:, 1])
            
            #auc_dict[classifier].append(roc_auc_score(y_test, probs))
            #precision, recall, _ = precision_recall_curve(y_test, probs)
            
            #auprc_dict[classifier].append(auc(recall, precision))
            
            #Update features table
            features_pred_dfs[classifier] = features_pred_dfs[classifier].append(X_test)
            pred_idx += 1
            
            print "Expected Variance = "+str(e)
            print "Mean Squared Error = "+str(m)
            print "R2 = "+str(r)

        #avg_auc = np.sum(auc_dict[classifier])/10.0
        print "avg expected variance = "+str(np.mean(exp_var))
        
        #avg_auprc = np.sum(auprc_dict[classifier])/10.0
        print "avg mean squared error = "+str(np.mean(mse))
        
        print "avg r2 = "+str(np.mean(r2))
            
        print "Finished "+ligand+" "+classifier
        break
    
    return features_pred_dfs

In [14]:
def combine_features_predictions(ligand, ordered_features, pred_df):
    
    pred_res = pred_df.copy(deep=True)
    for classifier in classifiers.keys():
        model_pred = pred_res[pred_res["model"] == classifier]
        model_pred.index = ordered_features[classifier].index
        
        #Creating the combined table
        features_pred = pd.concat([ordered_features[classifier], model_pred], axis=1)
        
        #Saving
        features_pred.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/mediode_NegAbs_NoFilter_NoDown_"+ligand+"_RF_features_pred.csv", sep='\t')
        break

### Predict

In [76]:
#%%time
    
for ligand in ligands:
    #Initialize dictionary
    pred_dict = defaultdict(list)
    auc_dict = defaultdict(list)
    auprc_dict = defaultdict(list)

    ordered_features = test_model(pred_dict, auc_dict, auprc_dict, ligands_features_df[ligand], ligands_negatives_df[ligand], ligand, "RandomUnderSampler")

    #pred_df = pd.DataFrame.from_dict(pred_dict)
    #auc_df = pd.DataFrame.from_dict(auc_dict)
    #auprc_df = pd.DataFrame.from_dict(auprc_dict)
    
    #Save to file
    #pred_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/mediode_NegAbs_NoFilter_NoDown_"+ligand+"_RF_0.1.csv", sep='\t')
    #auc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/mediode_NegAbs_NoFilter_NoDown_"+ligand+"_RF_0.1_auc.csv", sep='\t')
    #auprc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/mediode_NegAbs_NoFilter_NoDown_"+ligand+"_RF_0.1_auprc.csv", sep='\t')

    #Combine features and pred results to a unified table
    #combine_features_predictions(ligand, ordered_features, pred_df)

    print "Finished ligand "+ligand

fold #: 1
Expected Variance = -1.33326011736
Mean Squared Error = 0.0200644147433
R2 = -3.10353773209
fold #: 2
Expected Variance = -1.55180874464
Mean Squared Error = 0.0223719762558
R2 = -3.54521232472
fold #: 3
Expected Variance = -1.25974836046
Mean Squared Error = 0.0208191731074
R2 = -3.03302883589
fold #: 4
Expected Variance = -1.10496906296
Mean Squared Error = 0.0194716082792
R2 = -2.77738280095
fold #: 5
Expected Variance = -1.42531135389
Mean Squared Error = 0.0200978800096
R2 = -3.27335552132
fold #: 6
Expected Variance = -1.22339670037
Mean Squared Error = 0.0205221375852
R2 = -3.16028121495
fold #: 7
Expected Variance = -0.932893422106
Mean Squared Error = 0.0179669906655
R2 = -2.29118128165
fold #: 8
Expected Variance = -1.10813457224
Mean Squared Error = 0.0203422621806
R2 = -2.83703034321
fold #: 9
Expected Variance = -1.69595000959
Mean Squared Error = 0.0213938235071
R2 = -4.16857675438
fold #: 10
Expected Variance = -1.25399689644
Mean Squared Error = 0.020052744934

Expected Variance = -0.616608900739
Mean Squared Error = 0.0243255007717
R2 = -3.11726436335
fold #: 3
Expected Variance = 0.0918486983945
Mean Squared Error = 0.0211575062892
R2 = -1.30745974346
fold #: 4
Expected Variance = -0.148622713892
Mean Squared Error = 0.0229364571312
R2 = -1.86336875688
fold #: 5
Expected Variance = -0.040807591105
Mean Squared Error = 0.0219877748676
R2 = -1.41787861149
fold #: 6
Expected Variance = 0.0489725617143
Mean Squared Error = 0.0204966536929
R2 = -1.44160702626
fold #: 7
Expected Variance = -0.064090467365
Mean Squared Error = 0.02199914428
R2 = -1.75580282904
fold #: 8
Expected Variance = -0.011476865572
Mean Squared Error = 0.023204259019
R2 = -1.4928673652
fold #: 9
Expected Variance = -0.273313806658
Mean Squared Error = 0.0223493057613
R2 = -2.17121835088
fold #: 10
Expected Variance = -0.168552906482
Mean Squared Error = 0.0220169311663
R2 = -1.69304204385
avg expected variance = -0.150642660979
avg mean squared error = 0.0222981040038
avg r

In [63]:
def test_model_bin(pred_dict, auc_dict, auprc_dict, ligand_bind_features, ligand_negatives_features, ligand_name, downsample_method, features=[]):
    """
    Test different models in 10-folds cross-validation.
    """
    
    #Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
        
    #Arranging the features table by the CV order, for each model
    features_pred_dfs = dict.fromkeys(classifiers.keys())
    
    models_req_scaling = ["SVM", "KNN"]
    
    for classifier in classifiers.keys():
        model = classifiers[classifier]
        features_pred_dfs[classifier] = pd.DataFrame()
        
        #Create X and y and remove domain names
        X = pd.concat([ligand_bind_features.iloc[:,features], ligand_negatives_features.iloc[:,features]])
        X = X.drop('domain_name', 1)
        cols = X.columns
        
        if (classifier in models_req_scaling):
            idx = X.index
            cols = X.columns
            X = pd.DataFrame(scale(X)) #Is z-scoring the data needed?
            X.index = idx #Restoring indices after scaling
            X.columns = cols

        y = [1] * ligand_bind_features.shape[0]
        y.extend([0] * ligand_negatives_features.shape[0])
        y = np.array(y)

        binding_skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
        pred_idx = 1
        
        for train_index, test_index in binding_skf.split(X, y):
            print "fold #: "+str(pred_idx)
            X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
            y_train, y_test = y[train_index], y[test_index]

            #Down-sample negative examples to have balanced classes
            if (downsample_method == "NoDown"):
                X_train_sampled = X_train
                y_train_sampled = y_train
            else:
                if (downsample_method == "InstanceHardnessThreshold"):
                    downsampler = downsamplers[downsample_method][classifier]
                else:
                    downsampler = downsamplers[downsample_method]

                X_train_sampled, y_train_sampled = downsampler.fit_sample(X_train, y_train)
            
            # Adjust x/y train/test to be the continuous binding scores
            included_features_idx = [c in features_cols for c in cols]
            ligand_bind_idx = ligand_name+"_binding_score" == cols
            
            y_train_sampled = X_train_sampled[:,ligand_bind_idx]
            y_train_sampled = np.array(y_train_sampled).ravel()
            X_train_sampled = X_train_sampled[:,included_features_idx]
            
            y_test = X_test.loc[:,ligand_bind_idx]
            y_test = np.array(y_test).ravel()
            X_test = X_test.loc[:,included_features_idx]
            
            #fit to training data
            model = classifiers[classifier]
            model.fit(X_train_sampled, y_train_sampled)
            probs_list = []

            #probs = model.predict(X_test)
            #probs_list = probs
            
            pred_list = model.predict(X_test)
            #for l in probs:
            #    probs_list.append(l[1])
                
            pred_dict["obs"].extend(y_test)
            pred_dict["prob"].extend(pred_list)
            fold_list = [pred_idx] * len(pred_list)
            pred_dict["fold"].extend(fold_list)

            model_list = [classifier] * len(pred_list)
            pred_dict["model"].extend(model_list)
            
            #Bin testing labels
            y_test = (y_test >= 0.1).astype(int)
            
            #Update auc auprc dictionaries
            auc_dict[classifier].append(roc_auc_score(y_test, pred_list))
            precision, recall, _ = precision_recall_curve(y_test, pred_list)            
            auprc_dict[classifier].append(auc(recall, precision))
            
            #Update features table
            features_pred_dfs[classifier] = features_pred_dfs[classifier].append(X_test)
            pred_idx += 1
            
            print "AUC = "+str(auc_dict[classifier][-1])
            print "AUPRC = "+str(auprc_dict[classifier][-1])

        avg_auc = np.sum(auc_dict[classifier])/10.0
        print "avg auc = "+str(avg_auc)
        
        avg_auprc = np.sum(auprc_dict[classifier])/10.0
        print "avg auprc = "+str(avg_auprc)
            
        print "Finished "+ligand+" "+classifier
        break
    
    return features_pred_dfs

### Run as regression and bin to classify

In [64]:
#%%time
    
for ligand in ligands:
    #Initialize dictionary
    pred_dict = defaultdict(list)
    auc_dict = defaultdict(list)
    auprc_dict = defaultdict(list)

    ordered_features = test_model_bin(pred_dict, auc_dict, auprc_dict, ligands_features_df[ligand], ligands_negatives_df[ligand], ligand, "RandomUnderSampler")

    #pred_df = pd.DataFrame.from_dict(pred_dict)
    #auc_df = pd.DataFrame.from_dict(auc_dict)
    #auprc_df = pd.DataFrame.from_dict(auprc_dict)
    
    #Save to file
    #pred_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/mediode_NegAbs_NoFilter_NoDown_"+ligand+"_RF_0.1.csv", sep='\t')
    #auc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/mediode_NegAbs_NoFilter_NoDown_"+ligand+"_RF_0.1_auc.csv", sep='\t')
    #auprc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/mediode_NegAbs_NoFilter_NoDown_"+ligand+"_RF_0.1_auprc.csv", sep='\t')

    #Combine features and pred results to a unified table
    #combine_features_predictions(ligand, ordered_features, pred_df)

    print "Finished ligand "+ligand

fold #: 1
AUC = 0.924769319493
AUPRC = 0.243755555755
fold #: 2
AUC = 0.914100772343
AUPRC = 0.269401897546
fold #: 3
AUC = 0.893835969106
AUPRC = 0.265734329859
fold #: 4
AUC = 0.913254873115
AUPRC = 0.303841507762
fold #: 5
AUC = 0.915991173225
AUPRC = 0.202496549975
fold #: 6
AUC = 0.893299006988
AUPRC = 0.253324488076
fold #: 7
AUC = 0.907738139022
AUPRC = 0.297398836068
fold #: 8
AUC = 0.941956601692
AUPRC = 0.33234462428
fold #: 9
AUC = 0.934277307834
AUPRC = 0.317251803179
fold #: 10
AUC = 0.895696947407
AUPRC = 0.255759605334
avg auc = 0.913492011022
avg auprc = 0.274130919783
Finished dna RF
Finished ligand dna
fold #: 1
AUC = 0.817922794118
AUPRC = 0.0373979098786
fold #: 2
AUC = 0.923280617874
AUPRC = 0.0974089966258
fold #: 3
AUC = 0.895715336521
AUPRC = 0.181548681335
fold #: 4
AUC = 0.897717814212
AUPRC = 0.122659641074
fold #: 5
AUC = 0.857726331275
AUPRC = 0.180746921893
fold #: 6
AUC = 0.939490137628
AUPRC = 0.224715861079
fold #: 7
AUC = 0.84179555177
AUPRC = 0.181291